# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
import geopandas as gpd
import hvplot.pandas
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ciudad lazaro cardenas,17.9583,-102.2000,87.75,47,93,2.77,MX,1701884416
1,1,playa del carmen,20.6274,-87.0799,84.88,72,75,16.11,MX,1701884416
2,2,norcasia,5.5778,-74.8852,86.88,54,100,3.00,CO,1701884416
3,3,grytviken,-54.2811,-36.5092,32.99,79,87,22.21,GS,1701884416
4,4,qaqortoq,60.7167,-46.0333,39.34,55,100,19.64,GL,1701884416


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
import random

# Create a new column with sequential numbers as IDs
city_data_df['ID'] = range(len(city_data_df))

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(city_data_df, geometry=gpd.points_from_xy(city_data_df['Lng'], city_data_df['Lat']))


unique_cities = city_data_df['City'].unique()
city_color_mapping = {city: "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for city in unique_cities}

gdf['City Color'] = gdf['City'].map(city_color_mapping)

# Define the map plot with automatic coloring based on the 'ID' column

map_plot = gdf.hvplot(
    geo=True, 
    tiles='CartoLight', 
    size='Humidity', 
    color='City Color',  # Use the 'City Color' column for coloring
    hover=True,
    title='The humidity in each city with automatic coloring',
#     label=all_city_names  # Display all city names as labels
)


# Display the map plot
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City Color,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:

# Display the DataFrame after dropping rows with null values
city_data_df.keys()

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date', 'ID'],
      dtype='object')

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# Apply the conditions
ideal_conditions_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_conditions_df = ideal_conditions_df.dropna()

# Display the resulting DataFrame
print(ideal_conditions_df.head())


    City_ID        City      Lat     Lng  Max Temp  Humidity  Cloudiness  \
41       41  isafjordur  66.0755 -23.124     24.84        74           0   

    Wind Speed Country        Date  ID  
41        3.44      IS  1701884425  41  


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create a new DataFrame
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column for "Hotel Name"
hotel_df['Hotel Name'] = ""
hotel_df['Country'] = ""


# Display sample data from the new DataFrame
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ciudad lazaro cardenas,,17.9583,-102.2000,47,
1,playa del carmen,,20.6274,-87.0799,72,
2,norcasia,,5.5778,-74.8852,54,
3,grytviken,,-54.2811,-36.5092,79,
4,qaqortoq,,60.7167,-46.0333,55,


In [8]:
hotel_df.keys()

Index(['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name'], dtype='object')

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel

# Set the geographical coordinates
# latitude = 40.415392
# longitude = -3.7073743182788528


latitude = hotel_df['Lat'][1]
longitude = hotel_df['Lng'][1]


# Set the parameters for the type of search
limit = 20
radius = 10000
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

geoapify_key = '77d4415452f94c44b7d20b81eaefe9c5'
categories = f'accommodation.hotel'

# set up a parameters dictionary
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key,
    "categories":categories
    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
response = requests.get(base_url, params=params).json()

pprint(response)





{'features': [{'geometry': {'coordinates': [-87.07893299999999,
                                            20.626743100395622],
                            'type': 'Point'},
               'properties': {'address_line1': '2 HNOS',
                              'address_line2': 'Avenida 30 Norte, 77710 Playa '
                                               'del Carmen, ROO, Mexico',
                              'categories': ['accommodation',
                                             'accommodation.hotel',
                                             'building',
                                             'building.accommodation'],
                              'city': 'Playa del Carmen',
                              'country': 'Mexico',
                              'country_code': 'mx',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                       

In [31]:
import requests

# Set the parameters for the type of search
limit = 20
radius = 10000

geoapify_key = '77d4415452f94c44b7d20b81eaefe9c5'
categories = f'accommodation.hotel'

# set up a parameters dictionary
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":categories    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

response = requests.get(base_url, params=params).json()

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row['Lat']
    longitude = row['Lng']
#     pprint(latitude)
#     pprint(longitude)
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
#     pprint(params["filter"] )
#     pprint(params["bias"])  
#     break 

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    #     Make and API request using the params dictionaty
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['features'][0]['properties']['name']
        hotel_df.loc[index, 'Country'] = response['features'][0]['properties']['country']

    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        hotel_df.loc[index, "Country"] = "No Country found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    print(f"{hotel_df.loc[index, 'City']} - nearest Country: {hotel_df.loc[index, 'Country' ]}")
    


Starting hotel search
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
ciudad lazaro cardenas - nearest Country: Mexico
playa del carmen - nearest hotel: 2 HNOS
playa del carmen - nearest Country: Mexico
norcasia - nearest hotel: Hotel El Danubio
norcasia - nearest Country: Colombia
grytviken - nearest hotel: No hotel found
grytviken - nearest Country: No Country found
qaqortoq - nearest hotel: Hotel Qaqortoq
qaqortoq - nearest Country: Greenland
port-aux-francais - nearest hotel: Keravel
port-aux-francais - nearest Country: France
chonchi - nearest hotel: Hotel Huildín
chonchi - nearest Country: Chile
pingliang - nearest hotel: 平凉新都酒店
pingliang - nearest Country: China
adamstown - nearest hotel: No hotel found
adamstown - nearest Country: No Country found
port-gentil - nearest hotel: Tara-Mé
port-gentil - nearest Country: Gabon
pangkalanbuun - nearest hotel: Hotel Tiara
pangkalanbuun - nearest Country: Indonesia
jamestown - nearest hotel: DoubleTree Jamestown
jamestown -

jijiang - nearest hotel: No hotel found
jijiang - nearest Country: No Country found
minas de marcona - nearest hotel: No hotel found
minas de marcona - nearest Country: No Country found
santa lucia - nearest hotel: Hotel Rural Molino de Aqua
santa lucia - nearest Country: Spain
aswan - nearest hotel: Yaseen hotel
aswan - nearest Country: Egypt
udachny - nearest hotel: Вилюй
udachny - nearest Country: Russia
yatagan - nearest hotel: No hotel found
yatagan - nearest Country: No Country found
port mathurin - nearest hotel: Escale Vacances
port mathurin - nearest Country: Mauritius
koz'modem'yansk - nearest hotel: Лада
koz'modem'yansk - nearest Country: Russia
bokkos - nearest hotel: No hotel found
bokkos - nearest Country: No Country found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
haiku-pauwela - nearest Country: United States
maun - nearest hotel: Center Lodge Conference Center
maun - nearest Country: Botswana
enewetak - nearest hotel: No hotel found
enewetak - nearest Coun

salinopolis - nearest hotel: Hotel Salinópolis
salinopolis - nearest Country: Brazil
sitangkai - nearest hotel: No hotel found
sitangkai - nearest Country: No Country found
tamandare - nearest hotel: Pousada Baia dos Corais
tamandare - nearest Country: Brazil
cumaral - nearest hotel: Hotel San Francisco
cumaral - nearest Country: Colombia
charlottetown - nearest hotel: Holman Grand Hotel
charlottetown - nearest Country: Canada
klaksvik - nearest hotel: Hotel Klaksvik
klaksvik - nearest Country: Faroe Islands
colonia - nearest hotel: Wasserturm Hotel Cologne
colonia - nearest Country: Germany
maralal - nearest hotel: Peacock Rest House
maralal - nearest Country: Kenya
gazojak - nearest hotel: Жайхун
gazojak - nearest Country: Turkmenistan
bubaque - nearest hotel: Chez Julio
bubaque - nearest Country: Guinea-Bissau
laojunmiao - nearest hotel: No hotel found
laojunmiao - nearest Country: No Country found
kangding - nearest hotel: 溜溜城格桑花大酒店
kangding - nearest Country: China
wellington - ne

arco - nearest hotel: Hotel Al Sole
arco - nearest Country: Italy
sakaiminato - nearest hotel: かぐら旅館
sakaiminato - nearest Country: Japan
lata - nearest hotel: Residencial Botânico
lata - nearest Country: Portugal
kaka - nearest hotel: No hotel found
kaka - nearest Country: No Country found
ola - nearest hotel: No hotel found
ola - nearest Country: No Country found
zaporizhia - nearest hotel: Готель Театральний
zaporizhia - nearest Country: Ukraine
chanute - nearest hotel: No hotel found
chanute - nearest Country: No Country found
mariehamn - nearest hotel: Hotell Esplanad
mariehamn - nearest Country: Finland
dunda - nearest hotel: No hotel found
dunda - nearest Country: No Country found
otjiwarongo - nearest hotel: C'est Si Bon Hotel & Lodge
otjiwarongo - nearest Country: Namibia
kattivakkam - nearest hotel: No hotel found
kattivakkam - nearest Country: No Country found
la ronge - nearest hotel: La Ronge Hotel and Suites
la ronge - nearest Country: Canada
le port - nearest hotel: No h

andergrove - nearest hotel: No hotel found
andergrove - nearest Country: No Country found
harlingen - nearest hotel: Holiday Inn Express
harlingen - nearest Country: United States
puerto san jose - nearest hotel: Martita
puerto san jose - nearest Country: Guatemala
new plymouth - nearest hotel: Carrington Motel
new plymouth - nearest Country: New Zealand
la paz - nearest hotel: Alojamiento Tokio
la paz - nearest Country: Bolivia
tralee - nearest hotel: The Ashe Hotel
tralee - nearest Country: Ireland
bang lamung - nearest hotel: The Cop Seminar & Resort
bang lamung - nearest Country: Thailand
jabat - nearest hotel: No hotel found
jabat - nearest Country: No Country found
beloyarskiy - nearest hotel: Сибирь
beloyarskiy - nearest Country: Russia
arta - nearest hotel: Cronos
arta - nearest Country: Greece
saurimo - nearest hotel: Complexo Hoteleiro de Saurimo
saurimo - nearest Country: Angola
castillos - nearest hotel: A mi gente
castillos - nearest Country: Uruguay
whistler - nearest hot

turpan - nearest hotel: Oriental
turpan - nearest Country: China
kudahuvadhoo - nearest hotel: Hotel Riu Palace Maldivas
kudahuvadhoo - nearest Country: Maldives
szczecinek - nearest hotel: Hotel Pojezierze
szczecinek - nearest Country: Poland
surgut - nearest hotel: Факел
surgut - nearest Country: Russia
el vigia - nearest hotel: Posada
el vigia - nearest Country: Venezuela
san borja - nearest hotel: No hotel found
san borja - nearest Country: No Country found
masuguru - nearest hotel: No hotel found
masuguru - nearest Country: No Country found
jayapura - nearest hotel: Hotel Jasmine Jayapura
jayapura - nearest Country: Indonesia
fernie - nearest hotel: Royal Hotel
fernie - nearest Country: Canada
egilsstadir - nearest hotel: Valaskjálf
egilsstadir - nearest Country: Iceland
douz - nearest hotel: Louage station Douz
douz - nearest Country: Tunisia
palikir - national government center - nearest hotel: Mangrove Bay Hotel
palikir - national government center - nearest Country: Federated 

In [34]:
# Display sample data
hotel_df=hotel_df.dropna()
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ciudad lazaro cardenas,Mexico,17.9583,-102.2000,47,Hotel Sol del Pacífico
1,playa del carmen,Mexico,20.6274,-87.0799,72,2 HNOS
2,norcasia,Colombia,5.5778,-74.8852,54,Hotel El Danubio
3,grytviken,No Country found,-54.2811,-36.5092,79,No hotel found
4,qaqortoq,Greenland,60.7167,-46.0333,55,Hotel Qaqortoq


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:

# GeoDataFrame 생성
gdf = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df['Lng'], hotel_df['Lat']))


# Define the map plot
map_plot2 = gdf.hvplot(geo=True, tiles=' hotel name and the country', size=20, color='blue', hover=True)

# Display the map plot2

map_plot2


ValueError:  hotel name and the country tiles not recognized, must be one of: ['CartoDark', 'CartoEco', 'CartoLight', 'CartoMidnight', 'EsriAntarcticImagery', 'EsriArcticImagery', 'EsriArcticOceanBase', 'EsriArcticOceanReference', 'EsriDelormeWorldBaseMap', 'EsriImagery', 'EsriNatGeo', 'EsriOceanBase', 'EsriOceanReference', 'EsriReference', 'EsriTerrain', 'EsriUSATopo', 'EsriWorldBoundariesAndPlaces', 'EsriWorldBoundariesAndPlacesAlternate', 'EsriWorldDarkGrayBase', 'EsriWorldDarkGrayReference', 'EsriWorldHillshade', 'EsriWorldHillshadeDark', 'EsriWorldLightGrayBase', 'EsriWorldLightGrayReference', 'EsriWorldNavigationCharts', 'EsriWorldPhysical', 'EsriWorldShadedRelief', 'EsriWorldStreetMap', 'EsriWorldTopo', 'EsriWorldTransportation', 'OSM', 'OpenTopoMap'] or a tile object

In [ ]:
# Define the map plot with automatic coloring based on index
map_plot = gdf.hvplot(
    geo=True, 
    tiles='CartoLight', 
    size=20, 
    color='index',  # Use the index for automatic coloring
    hover=True,
    hover_cols=['City', 'Hotel Name', 'Country'],
    title='Hotel Locations on Map with Automatic Coloring'
)

# Display the map plot
map_plot